In [2]:
! pip install bokeh==2.4.3

In [3]:
! pip install backtesting

In [4]:
import yfinance as yf
import pandas as pd
from prophet import Prophet
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import ParameterGrid
from backtesting import Backtest, Strategy
from backtesting.test import SMA

/usr/local/lib/python3.10/dist-packages/backtesting/_plotting.py:50: UserWarning: Jupyter Notebook detected. Setting Bokeh output to notebook. This may not work in Jupyter clients without JavaScript support (e.g. PyCharm, Spyder IDE). Reset with `backtesting.set_bokeh_output(notebook=False)`.
  warnings.warn('Jupyter Notebook detected. '
/usr/local/lib/python3.10/dist-packages/backtesting/test/__init__.py:8: FutureWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  return pd.read_csv(join(dirname(__file__), filename),
/usr/local/lib/python3.10/dist-packages/backtesting/test/__init__.py:8: FutureWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-cons

In [5]:
# Pobieranie danych historycznych dla MSFT
ticker = 'MSFT'
data = yf.download(ticker, start='2020-01-01', end='2024-01-01')
data.reset_index(inplace=True)
data = data[['Date', 'Close']]
data.columns = ['ds', 'y']

[*********************100%%**********************]  1 of 1 completed


In [6]:
# Parametry do optymalizacji
param_grid = {
    'changepoint_prior_scale': [0.001, 0.01, 0.1, 0.5],
    'seasonality_prior_scale': [0.01, 0.1, 1.0, 10.0],
}

In [7]:
# Wybór najlepszych parametrów do modelu
best_params = None
best_mae = float('inf')

for params in ParameterGrid(param_grid):
    model = Prophet(**params, daily_seasonality=True)
    model.fit(data)

    future = model.make_future_dataframe(periods=126)  # Zakładając prognozowanie na 2024-01-01 do 2024-05-06
    forecast = model.predict(future)

    forecast_actual = forecast[['ds', 'yhat']].merge(data, on='ds', how='inner')
    mae = mean_absolute_error(forecast_actual['y'], forecast_actual['yhat'])

    if mae < best_mae:
        best_mae = mae
        best_params = params

print(f"Best parameters: {best_params}")

DEBUG:cmdstanpy:input tempfile: /tmp/tmpfdvjftg4/7ssidu0e.json
DEBUG:cmdstanpy:input tempfile: /tmp/tmpfdvjftg4/xxy_ncgj.json
DEBUG:cmdstanpy:idx 0
DEBUG:cmdstanpy:running CmdStan, num_threads: None
DEBUG:cmdstanpy:CmdStan args: ['/usr/local/lib/python3.10/dist-packages/prophet/stan_model/prophet_model.bin', 'random', 'seed=25021', 'data', 'file=/tmp/tmpfdvjftg4/7ssidu0e.json', 'init=/tmp/tmpfdvjftg4/xxy_ncgj.json', 'output', 'file=/tmp/tmpfdvjftg4/prophet_model8uiuhmpk/prophet_model-20240604153521.csv', 'method=optimize', 'algorithm=lbfgs', 'iter=10000']
15:35:21 - cmdstanpy - INFO - Chain [1] start processing
INFO:cmdstanpy:Chain [1] start processing
15:35:21 - cmdstanpy - INFO - Chain [1] done processing
INFO:cmdstanpy:Chain [1] done processing
DEBUG:cmdstanpy:input tempfile: /tmp/tmpfdvjftg4/texiukzh.json
DEBUG:cmdstanpy:input tempfile: /tmp/tmpfdvjftg4/iqwwuld0.json
DEBUG:cmdstanpy:idx 0
DEBUG:cmdstanpy:running CmdStan, num_threads: None
DEBUG:cmdstanpy:CmdStan args: ['/usr/local/

Best parameters: {'changepoint_prior_scale': 0.5, 'seasonality_prior_scale': 10.0}


In [8]:
# Stworzenie modelu z najlepszymi parametrami
model = Prophet(**best_params, daily_seasonality=True)
model.fit(data)
future = pd.date_range(start='2024-01-02', end='2024-05-02')
future_df = pd.DataFrame(future, columns=['ds'])
forecast = model.predict(future_df)
forecast

DEBUG:cmdstanpy:input tempfile: /tmp/tmpfdvjftg4/coqgg1aw.json
DEBUG:cmdstanpy:input tempfile: /tmp/tmpfdvjftg4/j_3gop3_.json
DEBUG:cmdstanpy:idx 0
DEBUG:cmdstanpy:running CmdStan, num_threads: None
DEBUG:cmdstanpy:CmdStan args: ['/usr/local/lib/python3.10/dist-packages/prophet/stan_model/prophet_model.bin', 'random', 'seed=51760', 'data', 'file=/tmp/tmpfdvjftg4/coqgg1aw.json', 'init=/tmp/tmpfdvjftg4/j_3gop3_.json', 'output', 'file=/tmp/tmpfdvjftg4/prophet_modeldt7wwhxs/prophet_model-20240604153549.csv', 'method=optimize', 'algorithm=lbfgs', 'iter=10000']
15:35:49 - cmdstanpy - INFO - Chain [1] start processing
INFO:cmdstanpy:Chain [1] start processing
15:35:50 - cmdstanpy - INFO - Chain [1] done processing
INFO:cmdstanpy:Chain [1] done processing


,ds,trend,yhat_lower,yhat_upper,trend_lower,trend_upper,additive_terms,additive_terms_lower,additive_terms_upper,daily,...,weekly,weekly_lower,weekly_upper,yearly,yearly_lower,yearly_upper,multiplicative_terms,multiplicative_terms_lower,multiplicative_terms_upper,yhat
0,2024-01-02,368.171141,370.003386,389.314368,368.171141,368.171141,11.299218,11.299218,11.299218,20.834189,...,0.764641,0.764641,0.764641,-10.299612,-10.299612,-10.299612,0.0,0.0,0.0,379.470360
1,2024-01-03,368.591999,369.988620,390.285883,368.591999,368.591999,11.056706,11.056706,11.056706,20.834189,...,1.181006,1.181006,1.181006,-10.958488,-10.958488,-10.958488,0.0,0.0,0.0,379.648706
2,2024-01-04,369.012858,369.446026,388.663044,369.012858,369.012858,10.307183,10.307183,10.307183,20.834189,...,1.135375,1.135375,1.135375,-11.662381,-11.662381,-11.662381,0.0,0.0,0.0,379.320040
3,2024-01-05,369.433716,369.648818,388.817289,369.433716,369.433716,9.649453,9.649453,9.649453,20.834189,...,1.214416,1.214416,1.214416,-12.399151,-12.399151,-12.399151,0.0,0.0,0.0,379.083169
4,2024-01-06,369.854574,365.304362,384.650563,369.854574,369.854574,5.074800,5.074800,5.074800,20.834189,...,-2.604273,-2.604273,-2.604273,-13.155115,-13.155115,-13.155115,0.0,0.0,0.0,374.929374
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
117,2024-04-28,417.411541,383.301292,518.587303,347.100078,480.684382,36.460414,36.460414,36.460414,20.834189,...,-2.604273,-2.604273,-2.604273,18.230498,18.230498,18.230498,0.0,0.0,0.0,453.871955
118,2024-04-29,417.832399,385.114389,526.477413,346.851490,481.844572,39.967548,39.967548,39.967548,20.834189,...,0.913108,0.913108,0.913108,18.220251,18.220251,18.220251,0.0,0.0,0.0,457.799947
119,2024-04-30,418.253257,383.346436,523.602893,346.615642,483.098143,39.813492,39.813492,39.813492,20.834189,...,0.764641,0.764641,0.764641,18.214662,18.214662,18.214662,0.0,0.0,0.0,458.066749
120,2024-05-01,418.674116,384.189936,524.664341,346.024394,484.380602,40.233249,40.233249,40.233249,20.834189,...,1.181006,1.181006,1.181006,18.218055,18.218055,18.218055,0.0,0.0,0.0,458.907365


In [9]:
# Generowanie sygnałów handlowych
def generate_signals(forecast, threshold=0.1):
    forecast['signal'] = 0
    forecast['signal'][forecast['yhat'].diff() > threshold] = 1
    forecast['signal'][forecast['yhat'].diff() < -threshold] = -1
    return forecast

signals = generate_signals(forecast)
print(signals)

            ds       trend  yhat_lower  yhat_upper  trend_lower  trend_upper  \
0   2024-01-02  368.171141  370.003386  389.314368   368.171141   368.171141   
1   2024-01-03  368.591999  369.988620  390.285883   368.591999   368.591999   
2   2024-01-04  369.012858  369.446026  388.663044   369.012858   369.012858   
3   2024-01-05  369.433716  369.648818  388.817289   369.433716   369.433716   
4   2024-01-06  369.854574  365.304362  384.650563   369.854574   369.854574   
..         ...         ...         ...         ...          ...          ...   
117 2024-04-28  417.411541  383.301292  518.587303   347.100078   480.684382   
118 2024-04-29  417.832399  385.114389  526.477413   346.851490   481.844572   
119 2024-04-30  418.253257  383.346436  523.602893   346.615642   483.098143   
120 2024-05-01  418.674116  384.189936  524.664341   346.024394   484.380602   
121 2024-05-02  419.094974  386.055398  528.258794   345.297701   485.512651   

     additive_terms  additive_terms_low

<ipython-input-9-cc11e9b9171a>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  forecast['signal'][forecast['yhat'].diff() > threshold] = 1
<ipython-input-9-cc11e9b9171a>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  forecast['signal'][forecast['yhat'].diff() < -threshold] = -1


In [10]:
# Backtesting
class ProphetStrategy(Strategy):
    def init(self):
        self.forecast = signals.set_index('ds')

    def next(self):
        date = self.data.index[-1]

        if date in self.forecast.index:
            signal = self.forecast.loc[date]['signal']

            if signal == 1:
                self.buy(sl=self.data.Close[-1] * 0.95, tp=self.data.Close[-1] * 1.06)  # Stop-loss 5%, Take-profit 6%
            elif signal == -1:
                self.sell(sl=self.data.Close[-1] * 1.05, tp=self.data.Close[-1] * 0.95)  # Stop-loss 5%, Take-profit 5%

In [11]:
# Przygotowanie danych do backtestu
bt_data = yf.download(ticker, start='2024-01-02', end='2024-05-02', interval='1d')

bt = Backtest(bt_data, ProphetStrategy, cash=10000)
stats = bt.run()

[*********************100%%**********************]  1 of 1 completed


In [12]:
bt.plot()

Row(id='1407', ...)

In [13]:
print(stats)

Start                     2024-01-02 00:00:00
End                       2024-05-01 00:00:00
Duration                    120 days 00:00:00
Exposure Time [%]                   97.619048
Equity Final [$]                 10207.048867
Equity Peak [$]                  11896.876634
Return [%]                           2.070489
Buy & Hold Return [%]                6.490147
Return (Ann.) [%]                    6.340961
Volatility (Ann.) [%]               23.376011
Sharpe Ratio                         0.271259
Sortino Ratio                        0.383783
Calmar Ratio                          0.41221
Max. Drawdown [%]                  -15.382846
Avg. Drawdown [%]                   -3.232724
Max. Drawdown Duration       41 days 00:00:00
Avg. Drawdown Duration       13 days 00:00:00
# Trades                                    5
Win Rate [%]                             40.0
Best Trade [%]                       6.625645
Worst Trade [%]                     -4.745621
Avg. Trade [%]                    